In [5]:
# ============================================
# 1) 설치
# ============================================
!pip -q install langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers pypdf


**MultiVectorRetriever**

원문에서
   > child를 여러 부분(예, 목차, 본문, 키워드 등) 에서 찾아서
     
    parent 청크에 연결해. 부모 chunk 내용을 결과로 도출

    Parent <> Child 연결은 "doc_id"로 되어 있음


In [ ]:

# ============================================
# 2) PDF 다운로드 (경제금융용어 700선)
# ============================================
import os, requests, urllib.request

url = "https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf"


pdf_path = "2020_경제금융용어 700선_게시.pdf"

# 우선 urlretrieve 시도
try:
    urllib.request.urlretrieve(url, filename=pdf_path)
except Exception:
    pass

# 비어있으면 requests로 재시도
if not os.path.exists(pdf_path) or os.path.getsize(pdf_path) == 0:
    r = requests.get(url, stream=True, timeout=60)
    r.raise_for_status()
    with open(pdf_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
print("✅ PDF ready:", pdf_path)

# ============================================
# 3) PDF 로드 & Parent/Child 청크 분할
#    (community 네임스페이스 사용)
# ============================================
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(f"📄 전체 페이지 수: {len(docs)}")

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=120)
child_splitter  = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=40)

parent_docs = parent_splitter.split_documents(docs)
print(f"🧩 부모 청크 수: {len(parent_docs)}")

# ============================================
# 4) 한국어 임베딩 준비 (community 네임스페이스)
# ============================================
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

# ============================================
# 5) 빈 FAISS 인덱스 + InMemoryStore 생성
#    (community 네임스페이스 + dependable_faiss_import)
# ============================================
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import dependable_faiss_import
from langchain.docstore import InMemoryDocstore  # 여기는 core 경로 유지

faiss = dependable_faiss_import()
dim = len(embedding.embed_query("테스트"))  # 임베딩 차원 확인
index = faiss.IndexFlatL2(dim)
vectorstore = FAISS(embedding.embed_query, index, InMemoryDocstore(), {})

# 부모 문서 저장소 (KV 스토어)
try:
    from langchain.storage import InMemoryStore   # 최신
except ImportError:
    class InMemoryStore(dict):                    # 폴백
        def mset(self, items):
            for k, v in items:
                self[k] = v
docstore = InMemoryStore()

# ============================================
# 6) MultiVectorRetriever 생성
#    (최신: langchain.retrievers, 폴백: community)
# ============================================
try:
    from langchain.retrievers import MultiVectorRetriever
except ImportError:
    from langchain_community.retrievers import MultiVectorRetriever

from uuid import uuid4
from langchain.schema import Document

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key="doc_id",
)

# ============================================
# 7) 부모-자식 매핑 (Child 청크만 색인)
# ============================================
print(" 부모-자식 매핑 중...")

parent_id_to_doc = []
child_texts, child_metas = [], []

for pdoc in parent_docs:
    pid = str(uuid4())

    # 부모 문서 저장
    parent_doc = Document(page_content=pdoc.page_content, metadata=pdoc.metadata | {"doc_id": pid})
    parent_id_to_doc.append((pid, parent_doc))

    # 자식 청크 생성 & 벡터 색인용 데이터 수집
    child_chunks = child_splitter.split_documents([pdoc])
    for c in child_chunks:
        child_texts.append(c.page_content)
        child_metas.append({"doc_id": pid, **(c.metadata or {})})

# 부모 저장
docstore.mset(parent_id_to_doc)

# 자식 벡터 색인
if child_texts:
    retriever.vectorstore.add_texts(texts=child_texts, metadatas=child_metas)

print("✅ 색인 완료")
print(f"부모 문서 수: {len(parent_id_to_doc)} | 자식 청크 수: {len(child_texts)}")

# ============================================
# 8) 검색 테스트
# ============================================
def ask(q, k=3):
    hits = retriever.get_relevant_documents(q)
    print(f"\n🔎 Q: {q}\n--- Top-{min(k,len(hits))} 부모 문서 ---")
    for i, d in enumerate(hits[:k], 1):
        page = d.metadata.get("page", "?")
        preview = d.page_content[:160].replace("\n"," ")
        print(f"{i:>2}. p.{page} | {preview}...")

# 테스트 질문
ask("인플레이션의 정의와 원인, 그리고 기준금리와의 관계는 무엇인가요?")
ask("환율 변동이 물가에 어떤 영향을 미치나요?")
ask("스태그플레이션의 의미와 정책 대응상의 어려움은 무엇인가요?")


✅ PDF ready: 2020_경제금융용어 700선_게시.pdf
📄 전체 페이지 수: 371
🧩 부모 청크 수: 466


⏳ 부모-자식 매핑 중...
✅ 색인 완료
부모 문서 수: 466 | 자식 청크 수: 1378

🔎 Q: 인플레이션의 정의와 원인, 그리고 기준금리와의 관계는 무엇인가요?
--- Top-3 부모 문서 ---
 1. p.334 | 318 경제금융용어 700선 여 매매할 때 이용된다. 과거에 프로그램매매는 지수차익거래 위주로 이루어졌으나 최근에 는 비차익거래가 대부분(2016년 상반기 중 거래대금 기준으로 98.3%)을 차지하고 있다.  이는 공모펀드 및 연기금(2010년)과 우정사업본부(2013년)에 대한 증권거...
 2. p.131 | 115 ㅁ  물가지수 시장에서 거래되는 여러 가지 상품과 서비스의 가격을 경제생활에서 차지하는 중요도 를 고려하여 평균한 종합적인 가격수준을 물가라고 하는데, 이 같은 물가의 변화를  한 눈에 알아볼 수 있도록 기준연도의 물가수준을 100으로 놓고 비교되는 다른 시점의  물가를 지수의 ...
 3. p.82 | 66 경제금융용어 700선 핵심지표중 하나이다. 기대인플레이션은 임금협상, 가격설정, 투자결정 등 경제주체의  의사결정에 반영되면서 최종적으로 실제 인플레이션에 영향을 미친다. 구체적인 경로를  살펴보면 기대인플레이션 상승 시 가계는 구매력 하락을 우려하여 명목임금 상승을  요구하게 되...


/tmp/ipython-input-3960166090.py:124: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  hits = retriever.get_relevant_documents(q)



🔎 Q: 환율 변동이 물가에 어떤 영향을 미치나요?
--- Top-3 부모 문서 ---
 1. p.197 | 181 ㅅ  인플레이션압력을 높일 가능성이 큰 상황으로 볼 수 있다. 따라서 물가안정을 주된  정책목표로 하는 중앙은행의 경우 실업률갭은 노동시장을 통한 인플레이션 상승압력을  평가하거나, 통화정책 기조가 노동시장 수급불균형 해소에 어느 정도 부합하는지 등을  살펴보는데 유용하게 활용할...
 2. p.131 | 115 ㅁ  물가지수 시장에서 거래되는 여러 가지 상품과 서비스의 가격을 경제생활에서 차지하는 중요도 를 고려하여 평균한 종합적인 가격수준을 물가라고 하는데, 이 같은 물가의 변화를  한 눈에 알아볼 수 있도록 기준연도의 물가수준을 100으로 놓고 비교되는 다른 시점의  물가를 지수의 ...
 3. p.332 | 316 경제금융용어 700선 자유변동환율제도중 어느 환율제도를 채택하느냐에 따라 구분된다. 즉 평가절상(또는 평가 절하)은 고정환율제도하에서 정부가 정책적 목적 등으로 자국통화의 대외가치인 환율을  인위적으로 일시에 조정하였을 경우 사용되며, 절상(또는 절하)은 일반적으로 자유변동환 율...

🔎 Q: 스태그플레이션의 의미와 정책 대응상의 어려움은 무엇인가요?
--- Top-3 부모 문서 ---
 1. p.185 | 169 ㅅ  스태그플레이션 제2차 세계대전 이전까지 경기침체기에는 물가가 하락하고 경기호황기에는 물가가  상승하는 것이 일반적 현상이었다. 그러나 제2차 세계대전 이후 특히 1970년대 들어  두 번의 유가파동을 겪으면서 실업이 늘어나는 침체기에도 인플레이션이 지속되는 현상 이 나타나기...
 2. p.158 | 부정적인 영향을 줄 가능성이 있다. 또한 사전적 정책방향의 제시내용이 지나치게 복잡 하고 이해하기 어려울 경우 오히려 정책운영 방식의 명료성을 제약하고 정책 불확실성이  다시 높아지는 문제가 발생할 수도 있다 .  연관검색어 : 양적완화정책, 제로금리정책, 통화정책 커뮤니케이션...
 3. p.125 | 109 

In [ ]:
import gradio as gr

# 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="경제금융용어 챗봇") # 챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message, 
                        callbacks=[ConsoleCallbackHandler()])
      bot_message = result['result']

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행
demo.launch(debug=True)